In [0]:
!pip install catboost

In [0]:
a = {0:7276,1:289829}
b= {0:7276}

In [0]:
list(b.items())[0][0]

0

In [0]:

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.externals import joblib
from collections import OrderedDict
from datetime import datetime
import glob
import numpy as np
import pandas as pd
import os
import itertools
from sklearn.metrics import r2_score
import sys
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.preprocessing import Imputer
from catboost import CatBoostClassifier,CatBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from collections import OrderedDict
from datetime import datetime
import glob
import numpy as np
import pandas as pd
import os
import itertools
import sys
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import lightgbm as lgb
from sklearn.preprocessing import Imputer, PolynomialFeatures, StandardScaler, OneHotEncoder, MinMaxScaler



import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')


import gc 
os.getcwd()
pd.options.display.max_columns = None


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/hackathons/Data Scientist Test/"

'Data Scientist Test.pdf'   dataset_00_with_header.csv	 lgb_1.pkl


In [0]:
filename = '/content/drive/My Drive/hackathons/Data Scientist Test/dataset_00_with_header.csv'

In [0]:
def nullfinder(df):
    for col in df:
        if sum(df[col].isnull()) > 0:
            print(f"For {col} number of nulls are {sum(df[col].isnull())}")

In [0]:

class datacleaner:
    def __init__(self, trainfile,targetcol,cat_threshold=100):
        #self.df_test = pd.read_csv(testfile,index_col=0)
        #print(self.df_test.columns)
        self.df_train = pd.read_csv(trainfile)  
        self.target = targetcol
        self.dfcolumns = self.df_train.columns.tolist()
        self.dfcolumns_nottarget = [col for col in self.dfcolumns if col != self.target]
        self.rejectcols = []
        self.retainedcols = []
        self.catcols = {}
        self.noncatcols = {}
        self.catcols_list = []
        self.noncatcols_list = []
        self.hightarge_corr_col = []
        self.threshold = cat_threshold
        
    def retail_reject_cols(self,threshold):
      def retail_reject_cols_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  #print('start')
                  for col in tmpdf:
                    if sum(tmpdf[col].isnull()) > 0:
                        percent_val = sum(tmpdf[col].isnull())/tmpdf[col].shape[0]
                        #print(f"For {col} number of nulls are {sum(df[col].isnull())} : {sum(df[col].isnull())/df[col].shape[0]}")
                        if percent_val > threshold:
                          self.rejectcols.append(col)
                  self.retainedcols = [col for col in tmpdf.columns.tolist() if col not in self.rejectcols]
                  print(f"Number of rejected columns {len(self.rejectcols)}")
                  print(f"Number of retained columns {len(self.retainedcols)}")
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return retail_reject_cols_lvl
    
    #def init_values(self,)
    def refresh_cat_noncat_cols_fn(self,tmpdf,threshold=100):
      try:
          self.catcols = {}
          self.catcols_list = []
          self.noncatcols = {}
          self.noncatcols_list = []
          self.dfcolumns = tmpdf.columns.tolist()
          self.dfcolumns_nottarget = [col for col in self.dfcolumns if col != self.target]
          for col in self.dfcolumns_nottarget:
            col_unique_cnt = tmpdf[col].nunique()
            if (col_unique_cnt < threshold) and ((tmpdf[col].dtype != 'float32') and (tmpdf[col].dtype != 'float64')):
              self.catcols.update({col:col_unique_cnt})
              self.catcols_list.append(col)
            else:
              self.noncatcols.update({col:col_unique_cnt})
              self.noncatcols_list.append(col)
      except:
          sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
        
    def refresh_cat_noncat_cols(self,threshold):
      def refresh_cat_noncat_cols_lvl1(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  self.catcols = {}
                  self.catcols_list = []
                  self.noncatcols = {}
                  self.noncatcols_list = []
                  self.dfcolumns = tmpdf.columns.tolist()
                  self.dfcolumns_nottarget = [col for col in self.dfcolumns if col != self.target]
                  for col in self.dfcolumns_nottarget:
                    col_unique_cnt = tmpdf[col].nunique()
                    if (col_unique_cnt < threshold) and ((tmpdf[col].dtype != 'float32') and (tmpdf[col].dtype != 'float64')):
                      self.catcols.update({col:col_unique_cnt})
                      self.catcols_list.append(col)
                    else:
                      self.noncatcols.update({col:col_unique_cnt})
                      self.noncatcols_list.append(col)
                  print('ref'+str(len(self.noncatcols_list)))
                  print('ref'+str(self.noncatcols_list))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return refresh_cat_noncat_cols_lvl1 
    
    def findcatcols(self,th):
      for col in self.dfcolumns_nottarget:
        col_unique_cnt = self.df_train[col].nunique()
        if col_unique_cnt < th:
          #print(f"{col} is category with unique values {df[col].nunique()}")
          self.catcols.update({col:col_unique_cnt})
        else:
          self.noncatcols.update({col:col_unique_cnt})
      print(f"Number of categorical column is {len(self.catcols.keys())}")
      print(f"Number of Non categorical column is {len(self.noncatcols.keys())}")
      return self.catcols,self.noncatcols

  
    def standardize_stratified(self, includestandcols=[]):
      def standardize_stratified_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  for col in tmpdf:
                    if tmpdf[col].dtype == 'float32' or tmpdf[col].dtype == 'float64' or (col in includestandcols):
                        tmpdf[col] = dtmpdff[col].replace(np.inf, 0.0)
                        if tmpdf[col].mean() > 1000:
                            scaler = MinMaxScaler(feature_range=(0,10))
                            tmpdf[col] = scaler.fit_transform(np.asarray(tmpdf[col]).reshape(-1,1))
                        elif tmpdf[col].mean() > 100:
                            scaler = MinMaxScaler(feature_range=(0,5))
                            #print(col)
                            tmpdf[col] = scaler.fit_transform(np.asarray(tmpdf[col]).reshape(-1,1))
                        elif tmpdf[col].mean() > 10:
                            scaler = MinMaxScaler(feature_range=(0,2))
                            #print(col)
                            tmpdf[col] = scaler.fit_transform(np.asarray(tmpdf[col]).reshape(-1,1))
                        else:
                            scaler = MinMaxScaler(feature_range=(0,1))
                            #print(col)
                            tmpdf[col] = scaler.fit_transform(np.asarray(tmpdf[col]).reshape(-1,1))
                        print("INFO : " + str(datetime.now()) + ' : ' + 'Column ' + col + 'is standardized')
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return standardize_stratified_lvl
    
    def featurization(self,cat_coltype=False):
      def featurization_lvl1(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  self.refresh_cat_noncat_cols_fn(tmpdf,self.threshold)
                  if cat_coltype:
                    column_list = self.catcols_list
                  else:
                    column_list = self.noncatcols_list
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe before featurization ' + str(tmpdf.shape))
                  for col in column_list:  
                    tmpdf[col+'_minus_mean'] = tmpdf[col] - np.mean(tmpdf[col])
                    tmpdf[col+'_minus_mean']  = tmpdf[col+'_minus_mean'].astype(np.float32)
                    tmpdf[col+'_minus_max'] = tmpdf[col] - np.max(tmpdf[col])
                    tmpdf[col+'_minus_max'] =  tmpdf[col+'_minus_max'].astype(np.float32)
                    tmpdf[col+'_minus_min'] = tmpdf[col] - np.min(tmpdf[col])
                    tmpdf[col+'_minus_min'] =  tmpdf[col+'_minus_min'].astype(np.float32)
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe after featurization ' + str(tmpdf.shape))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return featurization_lvl1
    
    def two_column_featurization(self,cat_coltype=False):
      def featurization_lvl1(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  self.refresh_cat_noncat_cols_fn(tmpdf,self.threshold)
                  if cat_coltype:
                    column_list = self.catcols_list
                  else:
                    column_list = self.noncatcols_list
                  #print('2col'+str(tmpdf.shape))
                  #print('2col'+str(tmpdf.columns))
                  #print('2col'+str(self.dfcolumns))
                  #print('2col'+str(column_list))
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe before two column featurization ' + str(tmpdf.shape))
                  for col in column_list:  
                    for col1 in column_list:
                      if col!=col1:
                        tmpdf[col+'_diff_'+col1]=tmpdf[col]-tmpdf[col1]
                        tmpdf[col+'_diff_'+col1] = tmpdf[col+'_diff_'+col1].astype(np.float32)
                        tmpdf[col+'_sum_'+col1]=tmpdf[col]+tmpdf[col1]
                        tmpdf[col+'_sum_'+col1] = tmpdf[col+'_sum_'+col1].astype(np.float32)
                        tmpdf[col+'_ratio_'+col1]=tmpdf[col]/tmpdf[col1]
                        tmpdf[col+'_ratio_'+col1] =  tmpdf[col+'_ratio_'+col1].astype(np.float32)
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe after two column featurization ' + str(tmpdf.shape))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return featurization_lvl1
    
    def standardize_simple(self,cat_coltype=False,range_tuple = (0,1)):
      def standardize_simple_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  self.refresh_cat_noncat_cols_fn(tmpdf,self.threshold)
                  if cat_coltype:
                    column_list = self.catcols_list
                  else:
                    column_list = self.noncatcols_list
                  scaler = MinMaxScaler(feature_range=range_tuple)
                  #tmpdf_col = [col for col in tmpdf.columns.tolist(),]
                  for col in column_list:  
                    tmpdf[col] = tmpdf[col].replace(np.inf, 0.0)
                    tmpdf[col] = scaler.fit_transform(np.asarray(tmpdf[col]).reshape(-1,1))
                    print("INFO : " + str(datetime.now()) + ' : ' + 'Column ' + col + 'is standardized')
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return standardize_simple_lvl
    
    def standardize_simple_auto(self,range_tuple = (0,1)):
      def standardize_simple_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  scaler = MinMaxScaler(feature_range=range_tuple)
                  #tmpdf_col = [col for col in tmpdf.columns.tolist(),]
                  for col in tmpdf:
                    if (tmpdf[col].dtype == 'float32') or (tmpdf[col].dtype == 'float64'):   
                      tmpdf[col] = tmpdf[col].replace(np.inf, 0.0)
                      tmpdf[col] = tmpdf[col].replace(np.nan, 0.0)
                      tmpdf[col] = tmpdf[col].replace(-np.inf, 0.0)
                      tmpdf[col] = scaler.fit_transform(np.asarray(tmpdf[col]).reshape(-1,1))
                      print("INFO : " + str(datetime.now()) + ' : ' + 'Column ' + col + 'is standardized')
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return standardize_simple_lvl
    
    def columnmapondf(self, col_list_of_dict = [['colname',{}],['colname',{}]]):
      def columnmapondf_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  for col in col_list_of_dict:
                      #print(mapdict)
                      #print(col)
                      convertedcolumns = tmpdf[col[0]].map(col[1])
                      tmpdf[col] = convertedcolumns
                      print("INFO : " + str(
                          datetime.now()) + ' : ' + 'Column ' + col[0] + ',mapped using dictionary : ' + str(col[1]))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return columnmapondf_lvl

    def dropcolumnfromdf(self, column_list=[]):
      def dropcolumnfromdf_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  tmpdf = tmpdf.drop(column_list, axis=1)
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Column ' + str(
                      column_list) + ',dropped from base dataframe')
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])

          return wrapper

      return dropcolumnfromdf_lvl
    
    def feature_importance(self,dfforimp,tobepredicted,modelname,featurelimit=0):
      colname = [col for col in dfforimp.columns.tolist() if col!=tobepredicted]
      X = dfforimp[colname]
      y = dfforimp[tobepredicted]
      if modelname =='rfclassifier':
        model = RandomForestClassifier(n_estimators=100, random_state =10)
      elif modelname == 'rfregressor':
        model = RandomForestRegressor(n_estimators=100, random_state =10)
      elif modelname =='lgbmclassifier':
        model = lgb.LGBMClassifier(n_estimators=1000, learning_rate=0.05, verbose=-1)
      elif modelname == 'lgbmregressor':
        model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05, verbose=-1)
      model.fit(X, y)
      feature_names = X.columns
      feature_importances = pd.DataFrame({'feature': feature_names, 'importance': model.feature_importances_})
      feature_importances = feature_importances.sort_values(by=['importance'], ascending=False).reset_index()
      feature_importances = feature_importances[['feature', 'importance']]
      if featurelimit == 0:
          return feature_importances
      else:
          return feature_importances[:featurelimit]

    def importantfeatures(self,dfforimp,tobepredicted,modelname='regressor',skipcols=[],featurelimit=0):
      f_imp = self.feature_importance(dfforimp, tobepredicted, modelname,featurelimit)
      allimpcols = list(f_imp['feature'])
      stuff = []
      for col in allimpcols:
          for skipcol in skipcols:
              if col != skipcol:
                  stuff.append(col)
              else:
                  pass
      return stuff,f_imp

    def convertdatatypes(self,cat_threshold = 100):
      def convertdatatypes_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  original_memory = tmpdf.memory_usage().sum()
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Size of dataset before applying converdatatypes '+str(original_memory))
                  for c in tmpdf:
                      if ('y' in c):
                          tmpdf[c] = tmpdf[c].fillna(0).astype(np.int32)
                      elif (tmpdf[c].dtype == 'object') and (tmpdf[c].nunique() < catthrshocat_thresholdld):
                          tmpdf[c] = tmpdf[c].astype('category')   
                      #elif (tmpdf[c].nunique() < cat_threshold):
                      #    tmpdf[c] = tmpdf[c].astype('category') 
                      elif tmpdf[c].dtype == float:
                          tmpdf[c] = tmpdf[c].astype(np.float32)
                      elif tmpdf[c].dtype == int:
                          tmpdf[c] = tmpdf[c].astype(np.int32)
                  new_memory = tmpdf.memory_usage().sum()
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Size of dataset after applying converdatatypes '+str(new_memory))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper

      return convertdatatypes_lvl

    def sumtwocolumns(self, combination_list=[['col1','col2','col1+col2']]):
      def sumtwocolumns_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  for col in combination_list:
                      tmpdf[col[2]] = tmpdf[col[0]] + tmpdf[col[1]]
                      print("INFO : " + str(datetime.now()) + ' : ' + 'Columns ' + col[0] + ' and ' + col[
                          1] + ' added and the name of new column is ' + str(totcol[0]))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])

          return wrapper

      return sumtwocolumns_lvl

    def difftwocolumns(self, combination_list=[['col1','col2','col1+col2']]):

      def difftwocolumns_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  for col in combination_list:
                      tmpdf[col[2]] = tmpdf[col[0]] + tmpdf[col[1]]
                      print("INFO : " + str(datetime.now()) + ' : ' + 'Columns ' + col[0] + ' and ' + col[
                          1] + ' subtracted and the name of new column is ' + str(totcol[0]))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])

          return wrapper

      return difftwocolumns_lvl

    def ohe_on_threshold(self, threshold,drop_converted_col=True):
      def ohe_on_threshold_lvl1(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  for col in tmpdf.columns.tolist():
                    if tmpdf[col].nunique() < threshold:
                      dummy = pd.get_dummies(tmpdf[col])
                      tmpdf = pd.concat([tmpdf, dummy], axis=1)
                      if drop_converted_col:
                        tmpdf = tmpdf.drop(col,axis=1)
                      print("INFO : " + str(datetime.now()) + ' : ' + 'Column ' + col + ' converted to dummies')
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return ohe_on_threshold_lvl1
    

    def remove_collinear(self, th=0.95):
      def converttodummies_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe before collinear drop ' + str(tmpdf.shape))
                  corr_matrix = tmpdf.corr().abs()
                  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
                  to_drop = [column for column in upper.columns if any(upper[column] > th)]
                  tmpdf  = tmpdf.drop(to_drop, axis=1)
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe after collinear drop ' + str(tmpdf.shape))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return converttodummies_lvl
    
    def high_coor_target_column(self,targetcol = 'y',th=0.5):
      def high_coor_target_column_lvl1(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe before retaining only highly corelated col with target ' + str(tmpdf.shape))
                  cols = [col for col in tmpdf.columns.tolist() if col != targetcol]
                  for col in cols:
                    tmpdcorr = tmpdf[col].corr(tmpdf[targetcol])
                    if tmpdcorr > th:
                      self.hightarge_corr_col.append(col)
                  cols = self.hightarge_corr_col + [targetcol]
                  tmpdf = tmpdf[cols]
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe after retaining only highly corelated col with target ' + str(tmpdf.shape))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return high_coor_target_column_lvl1      
    
    def ohe_on_column(self,drop_converted_col=True):
      def converttodummies_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  #self.refresh_cat_noncat_cols_fn(tmpdf,self.threshold)
                  column_list = self.catcols_list
                  for col in column_list:
                    dummy = pd.get_dummies(tmpdf[col])
                    dummy.columns = [col.lower()+'_'+str(x).lower().strip() for x in dummy.columns]
                    tmpdf = pd.concat([tmpdf, dummy], axis=1)
                    #tmpdf.columns = [col.lower()+'_'+str(x).lower().strip() for x in tmpdf.columns]
                    if drop_converted_col:
                      tmpdf = tmpdf.drop(col,axis=1)
                    print("INFO : " + str(datetime.now()) + ' : ' + 'Column ' + col + ' converted to dummies')
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return converttodummies_lvl
    
    def logtransform(self, logtransform_col=[]):
      def logtransform_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  for col in logtransform_col:
                      tmpdf[col] = tmpdf[col].apply(lambda x: np.log(x) if x != 0 else 0)
                      print("INFO : " + str(
                          datetime.now()) + ' : ' + 'Column ' + col + ' converted to corresponding log using formula: log(x)')
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])

          return wrapper

      return logtransform_lvl

    def applyformulaoncolumns(self, column_formula = []):
      def applyformulaoncolumns_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  for col, formula in column_formula.items():
                      formulabuilder = 'tmpdf[' + col + '].apply(lambda x: ' + formula + ' if x!=0 else 0)'
                      tmpdf[col] = eval(formulabuilder)
                      print("INFO : " + str(
                          datetime.now()) + ' : ' + 'Column ' + col + ' converted to corresponding log using formula:' + formula)
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper     
      return applyformulaoncolumns_lvl
        
        

    def applypca(self, cat_coltype = False,number_of_components = 50):
      def applypca_lvl1(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  self.refresh_cat_noncat_cols_fn(tmpdf,self.threshold)
                  if cat_coltype:
                    column_list = self.catcols_list
                  else:
                    column_list = self.noncatcols_list
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe before pca ' + str(tmpdf.shape))
                  non_included_cols = [col for col in self.dfcolumns_nottarget if col not in column_list]
                  non_included_cols = non_included_cols + [self.target]
                  x = tmpdf.loc[:, column_list].values
                  #y = tmpdf.loc[:,[self.target]].values
                  #x = StandardScaler().fit_transform(x)
                  pca = PCA(n_components=number_of_components)
                  #pca = PCA()
                  pca_comp = pca.fit_transform(x)
                  pca_df = pd.DataFrame(data = pca_comp, columns = ['pca_'+str(i) for i in range(number_of_components)])
                  #pca_df = pd.DataFrame(data = pca_comp)
                  #print(non_included_cols)
                  tmpdf = pd.concat([pca_df, tmpdf[non_included_cols]], axis = 1)
                  #print(tmpdf1.shape)
                  #print(non_included_cols)
                  #print(tmpdf1[non_included_cols].shape)
                  #tmpdf = pd.concat([tmpdf1,tmpdf[[self.target]]], axis = 1)  
                  self.explained_variance = pca.explained_variance_ratio_
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe after pca ' + str(tmpdf.shape))
                  gc.enable()
                  del x,pca_comp,pca_df
                  gc.collect()
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return applypca_lvl1

        
    def applytsne(self, cat_coltype = False,number_of_components = 50):
      def applytsne_lvl1(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  self.refresh_cat_noncat_cols_fn(tmpdf,self.threshold)
                  if cat_coltype:
                    column_list = self.catcols_list
                  else:
                    column_list = self.noncatcols_list
                  print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe before tsne ' + str(tmpdf.shape))
                  tmpdf_col = tmpdf.columns.tolist()
                  non_included_cols = [col for col in self.dfcolumns if col not in column_list]
                  x = tmpdf.loc[:, column_list].values
                  #y = tmpdf.loc[:,[self.target]].values
                  #x = StandardScaler().fit_transform(x)
                  tsne = TSNE(n_components=number_of_components)
                  tsne_comp = tsne.fit_transform(x)
                  tsne_df = pd.DataFrame(data = tsne_comp, columns = ['tsne_'+str(i) for i in range(number_of_components)])
                  tmpdf = pd.concat([tsne_df, tmpdf[non_included_cols]], axis = 1)
                  #print(tmpdf1.shape)
                  #tmpdf = pd.concat([tmpdf1,tmpdf[[self.target]]], axis = 1)
                  #print("INFO : " + str(datetime.now()) + ' : ' + 'Shape of dataframe after tsne ' + str(tmpdf.shape))
                  gc.enable()
                  del x,tsne_comp,tsne_df
                  gc.collect()
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
          return wrapper
      return applytsne_lvl1
        
    def convertnulls_to(self, columns_list=[],to_val=''):
      def convertnullsto_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  tmpdf.update(tmpdf[columns_list].fillna(to_val))
                  print("INFO : " + str(
                          datetime.now()) + ' : ' + 'Columns : ' + columns_list + ' converted to '+str(to_val))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])

          return wrapper

      return convertnullsto_lvl


    def addbackcolumn(self, stage):
      dictargname_col = 'addbackcolumn_col_' + str(stage)
      dictargname_map = 'addbackcolumn_map_' + str(stage)
      addbackcolumn_col = self.colnamelistdict[dictargname_col]
      addbackcolumn_map = self.maplistdict[dictargname_map]

      def addbackcolumn_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  for col in addbackcolumn_col:
                      if addbackcolumn_map == 'train':
                          tmpdf = pd.concat([tmpdf, self.df_train[col]], axis=1)
                      elif addbackcolumn_map == 'test':
                          tmpdf = pd.concat([tmpdf, self.df_test[col]], axis=1)
                      else:
                          print("INFO : " + str(
                              datetime.now()) + ' : ' + 'Unable to add column.Please enter test or train in input sheet')
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])

          return wrapper

      return addbackcolumn_lvl

    def testtrainna(self, tmpdf, tobepred_col):
      key = tmpdf[tobepred_col].isnull()
      df_na = tmpdf.loc[key]
      df_notNA_train_x = pd.DataFrame()
      df_notNA_train_y = pd.DataFrame()
      df_NA_test_x = pd.DataFrame()
      df_NA_test_y = pd.DataFrame()
      try:
          #print(df_na.shape)
          if df_na.empty:
              print(
                  "INFO : " + str(
                      datetime.now()) + ' : ' + 'Column ' + tobepred_col + 'has no null values so imputing cancelled')
              return df_notNA_train_x, df_notNA_train_y, df_NA_test_x, df_NA_test_y
          else:
              df_notNA = tmpdf.loc[~key]
              df_notNA_train_x = df_notNA.dropna(how='any')
              df_notNA_train_y = df_notNA_train_x[tobepred_col]
              df_notNA_train_x = df_notNA_train_x.drop(tobepred_col, axis=1)
              df_NA_test_y = df_na[tobepred_col]
              df_NA_test_x = df_na.drop(tobepred_col, axis=1)
              df_NA_test_x = df_NA_test_x.ffill().bfill()
              df_NA_test_x = df_NA_test_x.dropna(axis=1, how='all')
              df_notNA_train_x = df_notNA_train_x[df_NA_test_x.columns]
          return df_notNA_train_x, df_notNA_train_y, df_NA_test_x, df_NA_test_y
      except:
          sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])

    def model_null_impute_notcat_dt(self):
      def model_null_impute_notcat_dt_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  self.refresh_cat_noncat_cols_fn(tmpdf,self.threshold)
                  column_list = self.catcols_list
                  for col in column_list:
                      #print(col)
                      df_notNA_train_x, df_notNA_train_y, df_NA_test_x, df_NA_test_y = self.testtrainna(tmpdf, col)
                      if df_notNA_train_x.empty:
                          pass
                          #print("INFO : " + str(datetime.now()) + ' : ' + 'imputing cancelled')
                      else:
                          dtree = DecisionTreeRegressor()
                          #print(df_notNA_train_x.columns[df_notNA_train_x.isna().any()].tolist())
                          #print(df_notNA_train_y.columns[df_notNA_train_y.isna().any()].tolist())
                          #print(df_notNA_train_x.shape, df_notNA_train_y.shape)
                          dtree.fit(df_notNA_train_x, df_notNA_train_y)
                          predictions = dtree.predict(df_NA_test_x)
                          df_NA_test_x[col] = predictions
                          tmpdf[col].update(df_NA_test_x[col])
                          print("INFO : " + str(
                              datetime.now()) + ' : ' + 'Column ' + col + ',data imputing completed : No. of data imputed = ' + str(
                              df_NA_test_x.shape[0]))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])

          return wrapper

      return model_null_impute_notcat_dt_lvl

    def model_null_impute_cat_dt(self):
      def model_null_impute_notcat_dt_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  self.refresh_cat_noncat_cols_fn(tmpdf,self.threshold)
                  column_list = self.noncatcols_list
                  for col in column_list:
                      #print(col)
                      df_notNA_train_x, df_notNA_train_y, df_NA_test_x, df_NA_test_y = self.testtrainna(tmpdf, col)
                      if df_notNA_train_x.empty:
                          pass
                          #print("INFO : " + str(datetime.now()) + ' : ' + 'imputing cancelled')
                      else:
                          dtree = DecisionTreeClassifier()
                          #print(df_notNA_train_x.columns[df_notNA_train_x.isna().any()].tolist())
                          #print(df_notNA_train_y.columns[df_notNA_train_y.isna().any()].tolist())
                          #print(df_notNA_train_x.shape, df_notNA_train_y.shape)
                          dtree.fit(df_notNA_train_x, df_notNA_train_y)
                          predictions = dtree.predict(df_NA_test_x)
                          #print(predictions)
                          df_NA_test_x[col] = predictions
                          tmpdf[col].update(df_NA_test_x[col])
                          print("INFO : " + str(
                              datetime.now()) + ' : ' + 'Column ' + col + ',data imputing completed : No. of data imputed = ' + str(
                              df_NA_test_x.shape[0]))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])

          return wrapper

      return model_null_impute_notcat_dt_lvl
    
    
    
    
    def model_null_impute_notcat_rf(self):
      def model_null_impute_notcat_rf_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  self.refresh_cat_noncat_cols_fn(tmpdf,self.threshold)
                  column_list = self.noncatcols_list
                  for col in column_list:
                      #df_notNA_train_x, df_notNA_train_y, df_NA_test_x, df_NA_test_y = self.testtrainna(tmpdf, col)
                      df_notNA_train_x, df_notNA_train_y, df_NA_test_x, df_NA_test_y = self.testtrainna(tmpdf, col)
                      if df_notNA_train_x.empty:
                          pass
                          #print("INFO : " + str(datetime.now()) + ' : ' + 'imputing cancelled')
                      else:
                          rfc = RandomForestRegressor()
                          rfc.fit(df_notNA_train_x, df_notNA_train_y)
                          predictions = rfc.predict(df_NA_test_x)
                          df_NA_test_x[col] = predictions
                          tmpdf[col].update(df_NA_test_x[col])
                          print("INFO : " + str(
                              datetime.now()) + ' : ' + 'Column ' + col + ',data imputing completed : No. of data imputed = ' + str(
                              df_NA_test_x.shape[0]))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])

          return wrapper

      return model_null_impute_notcat_rf_lvl

    def model_null_impute_cat_rf(self):

      def model_null_impute_cat_rf_lvl(func):
          def wrapper(tmpdf):
              try:
                  tmpdf = func(tmpdf)
                  self.refresh_cat_noncat_cols_fn(tmpdf,self.threshold)
                  column_list = self.catcols_list
                  for col in column_list:
                      df_notNA_train_x, df_notNA_train_y, df_NA_test_x, df_NA_test_y = self.testtrainna(tmpdf, col)
                      if df_notNA_train_x.empty:
                          pass
                          #print("INFO : " + str(datetime.now()) + ' : ' + 'imputing cancelled')
                      else:
                          rfc = RandomForestClassifier()
                          rfc.fit(df_notNA_train_x, df_notNA_train_y)
                          predictions = rfc.predict(df_NA_test_x)
                          df_NA_test_x[col] = predictions
                          tmpdf[col].update(df_NA_test_x[col])
                          print("INFO : " + str(
                              datetime.now()) + ' : ' + 'Column ' + col + ',data imputing completed : No. of data imputed = ' + str(
                              df_NA_test_x.shape[0]))
                  return tmpdf
              except:
                  sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])

          return wrapper

      return model_null_impute_cat_rf_lvl


In [0]:
dc = datacleaner(filename,targetcol='y',cat_threshold=150)

In [0]:
@dc.model_null_impute_cat_rf()
@dc.model_null_impute_notcat_rf()
@dc.retail_reject_cols(threshold=0.4)
def clean_data(tmpdf):
  return tmpdf

In [3]:
a = 'True'
t = bool(a)
type(t)

bool

In [0]:
cleandf = clean_data(dc.df_train)

Number of rejected columns 23
Number of retained columns 282
INFO : 2019-08-16 21:07:12.902403 : Column x001has no null values so imputing cancelled
INFO : 2019-08-16 21:07:13.894577 : Column x002,data imputing completed : No. of data imputed = 21432
INFO : 2019-08-16 21:07:14.724998 : Column x003,data imputing completed : No. of data imputed = 21432
INFO : 2019-08-16 21:07:15.565748 : Column x004,data imputing completed : No. of data imputed = 21424
INFO : 2019-08-16 21:07:16.431398 : Column x005,data imputing completed : No. of data imputed = 6110
INFO : 2019-08-16 21:07:17.394329 : Column x041,data imputing completed : No. of data imputed = 36872
INFO : 2019-08-16 21:07:17.397614 : Column x042has no null values so imputing cancelled
INFO : 2019-08-16 21:07:17.407049 : Column x043has no null values so imputing cancelled
INFO : 2019-08-16 21:07:18.282872 : Column x044,data imputing completed : No. of data imputed = 19674
INFO : 2019-08-16 21:07:19.133283 : Column x045,data imputing co

In [0]:
f_imp_list,f_impdf = dc.importantfeatures(cleandf,tobepredicted='y',modelname='lgbmregressor',skipcols=[],featurelimit=0)

In [0]:
important_columns = f_impdf[:200]['feature'].tolist() + ['y']

In [0]:
#cleandf = cleandf[important_columns]

In [0]:
#@dc.refresh_cat_noncat_cols(threshold=100)
#@dc.standardize_simple_auto(range_tuple = (0,10))
#@dc.refresh_cat_noncat_cols(threshold=200)
@dc.ohe_on_column(drop_converted_col=True)
@dc.refresh_cat_noncat_cols(threshold=150)
#@dc.featurization(cat_coltype=False)
#@dc.two_column_featurization(cat_coltype=False)
#@dc.refresh_cat_noncat_cols(threshold=100)
#@dc.convertdatatypes(cat_threshold=150)
#@dc.applypca(cat_coltype=False ,number_of_components = 10)
#@dc.refresh_cat_noncat_cols(threshold=100)
#@dc.high_coor_target_column(targetcol = 'y',th=0.05)
#@dc.remove_collinear(th=0.95)
#@dc.model_null_impute_cat_rf()
#@dc.model_null_impute_notcat_rf()
#@dc.refresh_cat_noncat_cols(threshold=100)
@dc.convertdatatypes(cat_threshold=150)
#@dc.retail_reject_cols(threshold=0.4)
def cleandata_lgbm(tmpdf):
  return tmpdf

In [0]:
cleandf = cleandata_lgbm(cleandf)

INFO : 2019-08-16 21:10:00.971749 : Size of dataset before applying converdatatypes 244000080
INFO : 2019-08-16 21:10:12.426474 : Size of dataset after applying converdatatypes 122000080
ref80
ref['x001', 'x002', 'x003', 'x004', 'x005', 'x041', 'x042', 'x043', 'x044', 'x045', 'x057', 'x058', 'x075', 'x098', 'x111', 'x120', 'x121', 'x130', 'x131', 'x140', 'x141', 'x146', 'x148', 'x155', 'x162', 'x186', 'x194', 'x195', 'x196', 'x204', 'x205', 'x206', 'x214', 'x215', 'x216', 'x220', 'x221', 'x222', 'x223', 'x233', 'x234', 'x235', 'x236', 'x237', 'x238', 'x239', 'x242', 'x243', 'x253', 'x255', 'x256', 'x257', 'x258', 'x259', 'x264', 'x265', 'x266', 'x267', 'x268', 'x272', 'x273', 'x274', 'x275', 'x279', 'x280', 'x281', 'x287', 'x288', 'x289', 'x290', 'x291', 'x292', 'x293', 'x294', 'x295', 'x296', 'x297', 'x302', 'x303', 'x304']
INFO : 2019-08-16 21:10:13.120196 : Column x006 converted to dummies
INFO : 2019-08-16 21:10:13.328300 : Column x007 converted to dummies
INFO : 2019-08-16 21:10:1

In [0]:
import torch.utils.data as data_utils
import torch
def create_train_test_val_loaders(train,test,val=0.2,batch_size=50):
  targetcol = 'y'
  train,val = train_test_split(train,test_size=0.2,random_state=0)
  train  = train.reset_index(drop=True)
  test  = test.reset_index(drop=True)
  val  = val.reset_index(drop=True)
  feature_cols = [col for col in train.columns.tolist() if col!='y']
  #f_tensor = torch.tensor(train[feature_cols].values)
  #t_tensor = torch.tensor(train['y'].values)
  #RegressionColumnarDataset(df, catcols,targetcol,mode='train',catth=0.2)
  train_tensor = data_utils.TensorDataset(torch.tensor(train[feature_cols].values), torch.tensor(train['y'].values))
  train_loader = data_utils.DataLoader(train_tensor, batch_size=batch_size, shuffle=True)
  val_tensor = data_utils.TensorDataset(torch.tensor(val[feature_cols].values), torch.tensor(val['y'].values))
  val_loader = data_utils.DataLoader(val_tensor, batch_size=batch_size, shuffle=True)
  test_tensor = data_utils.TensorDataset(torch.tensor(test[feature_cols].values), torch.tensor(test['id'].values))
  test_loader = data_utils.DataLoader(test_tensor, batch_size=batch_size, shuffle=True)
  return train_loader,val_loader,test_loader
  

In [0]:
from sklearn.model_selection import train_test_split
cleandf['id'] = cleandf.index +1
train,test = train_test_split(cleandf,test_size=0.2,random_state=0)
train  = train.reset_index(drop=True)
test  = test.reset_index(drop=True)
train_loader,val_loader,test_loader = create_train_test_val_loaders(train,test,val=0.2,batch_size=500)

In [0]:
class LinearRegression(torch.nn.Module):
    def __init__(self, size_list,drops_list, use_bn=True, D_out=1):
        super().__init__()
        self.use_bn = use_bn
        self.linears = torch.nn.ModuleList([torch.nn.Linear(size_list[i], size_list[i+1]) for i in range(len(size_list)-1)])
        self.batch1d = torch.nn.ModuleList([torch.nn.BatchNorm1d(sz) for sz in size_list[1:]])
        for x in self.linears: 
          torch.nn.init.kaiming_normal_(x.weight.data)
        self.drops = torch.nn.ModuleList([torch.nn.Dropout(drop) for drop in drops_list])
        self.output = torch.nn.Linear(size_list[-1], D_out)
        torch.nn.init.kaiming_normal_(self.output.weight.data)

    def forward(self, x):
      for l,d,b in zip(self.linears, self.drops, self.batch1d):
        #x = torch.nn.functional.relu(l(x))
        x = l(x)
        if self.use_bn: 
          x = b(x)
        x = d(x)
      x = self.output(x)
      return x

In [0]:
def model_pytorch(model,criterion,optimizer,training_loader,validation_loader,device,epochs = 10):
    running_loss_history = []
    running_corrects_history = []
    val_running_loss_history = []
    val_running_corrects_history = []
    #batch = 0
    model.to(device)
    for e in range(epochs):
        batch = 0
        running_loss = 0.0
        running_corrects = 0.0
        val_running_loss = 0.0
        val_running_corrects = 0.0
  
        for inputs, labels in training_loader:
            inputs = inputs.to(device)
            #print(inputs.shape)
            labels = labels.to(device)
            inputs = inputs.float()
            labels = labels.float()
            #print(labels.shape)
            batch = batch + len(inputs)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            #print(outputs)
            #print(labels)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            preds = torch.abs(outputs.squeeze() - labels) <= 3
            #t = torch.sum(preds)
            #print(preds)
            #print(t)
            #print(torch.sum(preds))
            running_loss += loss.item()
            running_corrects += torch.sum(preds)
            
        else:
            valbatch = 0
            with torch.no_grad():
                for val_inputs, val_labels in validation_loader:
                    val_inputs = val_inputs.to(device)
                    val_labels = val_labels.to(device)
                    val_labels = val_labels.float()
                    val_inputs = val_inputs.float()
                    valbatch = valbatch + len(val_inputs)
                    val_outputs = model(val_inputs)          
                    val_loss = criterion(val_outputs, val_labels)
                    val_preds = torch.abs(val_outputs.squeeze() - val_labels) <= 3
                    val_running_loss += val_loss.item()
                    val_running_corrects += torch.sum(val_preds)
        epoch_loss = running_loss/len(training_loader)
        epoch_acc = running_corrects.float()/ batch
        running_loss_history.append(epoch_loss)
        running_corrects_history.append(epoch_acc)
        val_epoch_loss = val_running_loss/len(validation_loader)
        val_epoch_acc = val_running_corrects.float()/ valbatch
        val_running_loss_history.append(val_epoch_loss)
        val_running_corrects_history.append(val_epoch_acc)
        print('epoch :', (e+1))
        print('training loss: {:.4f}, acc {:.4f} '.format(epoch_loss, epoch_acc.item()))
        print('validation loss: {:.4f}, validation acc {:.4f} '.format(val_epoch_loss, val_epoch_acc.item()))
    return model

In [0]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = torch.nn.MSELoss()
        
    def forward(self,pred,actual):
        return torch.sqrt(self.mse(pred,actual))

In [0]:
lr=0.01
criterion = RMSELoss() 

inputDim =1
outputDim =1
inputval = cleandf.shape[1]-1
model = LinearRegression(size_list=[inputval,100,30,100,50,10],drops_list=[0.001,0.02,0.04,0.01,0.01], use_bn=True, D_out=1)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
epochs = 100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
model

LinearRegression(
  (linears): ModuleList(
    (0): Linear(in_features=7474, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=30, bias=True)
    (2): Linear(in_features=30, out_features=100, bias=True)
    (3): Linear(in_features=100, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=10, bias=True)
  )
  (batch1d): ModuleList(
    (0): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (drops): ModuleList(
    (0): Dropout(p=0.001)
    (1): Dropout(p=0.02)
    (2): Dropout(p=0.04)
    (3): Dropout(p=0.01)
    (4): Dropout(p=0.01)
  )
  (output): Linear

In [25]:
model = model_pytorch(model,criterion,optimizer,train_loader,val_loader,device,epochs = 500)

epoch : 1
training loss: 617.7017, acc 0.0000 
validation loss: 594.3564, validation acc 0.0000 
epoch : 2
training loss: 539.6104, acc 0.0000 
validation loss: 473.0209, validation acc 0.0000 
epoch : 3
training loss: 378.6033, acc 0.0002 
validation loss: 276.0556, validation acc 0.0016 
epoch : 4
training loss: 181.3769, acc 0.0133 
validation loss: 123.4724, validation acc 0.0144 
epoch : 5
training loss: 120.3614, acc 0.0140 
validation loss: 120.3182, validation acc 0.0145 
epoch : 6
training loss: 120.1435, acc 0.0136 
validation loss: 120.2133, validation acc 0.0134 
epoch : 7
training loss: 120.1050, acc 0.0142 
validation loss: 120.3365, validation acc 0.0128 
epoch : 8
training loss: 120.1137, acc 0.0138 
validation loss: 120.2321, validation acc 0.0131 
epoch : 9
training loss: 120.1195, acc 0.0135 
validation loss: 120.2612, validation acc 0.0137 
epoch : 10
training loss: 120.1103, acc 0.0141 
validation loss: 120.2639, validation acc 0.0132 
epoch : 11
training loss: 120

In [26]:
train.shape

(80000, 7475)

In [0]:
def predict_pytorch(model, test_loader):
  sub = pd.DataFrame(columns=['id','y_pred_pytorch'])
  with torch.no_grad():
    #model.eval()
    for inputs,idcol in test_loader:
        inputs = inputs.to(device)
        #print(inputs.squeeze())
        out = model(inputs.float()).squeeze().cpu().data.numpy()
        idcol = idcol.cpu().data.numpy()
        #print(out)
        submission = pd.DataFrame({'id': idcol, 'y_pred_pytorch': out})
        #print(submission)
        sub = pd.concat([sub,submission])
    return sub

In [0]:
sub1 = predict_pytorch(model, test_loader)

In [29]:
sub1.head()

,id,y_pred_pytorch
0,48035,623.910095
1,87636,623.682861
2,50791,623.769897
3,42914,623.658081
4,35235,623.753784


In [30]:
sub = pd.merge(sub1,cleandf[['id','y']],on='id',how='left')
sub['y_pred-y'] = (sub['y_pred_pytorch'] - sub['y']).abs()
sub['result'] = sub['y_pred-y'].apply(lambda x: 0 if x>3 else 1)
result_dict = dict(sub['result'].value_counts())
result_dict[1]/(result_dict[0]+result_dict[1])

0.014

In [31]:
sub.head()

,id,y_pred_pytorch,y,y_pred-y,result
0,48035,623.910095,510,113.910095,0
1,87636,623.682861,721,97.317139,0
2,50791,623.769897,398,225.769897,0
3,42914,623.658081,529,94.658081,0
4,35235,623.753784,815,191.246216,0


In [32]:
result_dict

{0: 19720, 1: 280}